### Verify the services are running


In [ ]:
#First let us verify that the IRSA backend service is running
!curl -k https://irsa-svc.domino-field/healthz

In [ ]:
!curl http://localhost:6003/healthz

In [ ]:
!cat $AWS_WEB_IDENTITY_TOKEN_FILE

In [ ]:
!cat $AWS_CONFIG_FILE

### Now lets get some action

Let us actual make calls using the `boto3` api now :


### Listing Profiles

Let us use the boto3 library to fetch all the profiles available. This is similar to `cat $AWS_CONFIG_FILE`

In [ ]:
import boto3.session
for profile in boto3.session.Session().available_profiles:
    print(profile)

### Listing the bucket

First lets verify what the identity of the profile is. This is determines by a combination of the web identity token
and the underlying assumed role in the Assests Account via the role in the EKS Account

In [ ]:
import boto3.session
list_bucket_profile_name='acme-list-bucket-role'
session = boto3.session.Session(profile_name=list_bucket_profile_name)
sts_client = session.client('sts')
sts_client.get_caller_identity()

In [ ]:
#Now list the bucket
import boto3
test_bucket='domino-acme-test-bucket'
list_bucket_profile_name='acme-list-bucket-role'
session = boto3.session.Session(profile_name=list_bucket_profile_name)
s3_client = session.client('s3')
for key in s3_client.list_objects(Bucket=test_bucket)['Contents']:
    print(key)

### Writing to the bucket

Now we write to the bucket. For that we need to use the profile 'acme-update-bucket-role'. This role in the assets
account will have the following policies attached to it (List, Read, Update permission)

1. `acme-list-bucket-policy` [Only the relevant snipper below]


2. `acme-read-bucket-policy` 

```json
        {
            "Effect": "Allow",
            "Action": ["s3:Get*"],
            "Resource": ["arn:aws:s3:::domino-acme-test-bucket/*"]
        }
```

2. `acme-update-bucket-policy` 

```json
     [
        {
            "Sid": "WriteObjectsInBucket",
            "Effect": "Allow",
            "Action": "s3:PutObject",
            "Resource": ["arn:aws:s3:::domino-acme-test-bucket/*"]
        },
        {
            "Sid": "DeleteObjectsInBucket",
            "Effect": "Allow",
            "Action": "s3:DeleteObject",
            "Resource": ["arn:aws:s3:::domino-acme-test-bucket/*"]
        }
    ]
```

Now let us create a object with key `$DOMINO_STARTING_USERNAME.txt`

In [ ]:
import boto3
import os
test_bucket='domino-acme-test-bucket'
starting_user = os.environ['DOMINO_STARTING_USERNAME']
update_bucket_profile_name='acme-update-bucket-role'
session = boto3.session.Session(profile_name=update_bucket_profile_name)
s3_client = session.client('s3')
object_data = "This is a random string."
object_key = f'{starting_user}.txt'
s3_client.put_object(Body=object_data, Bucket=test_bucket, Key=object_key)


### Reading a key from the bucket

Let us now read the newly added object from the bucket. This is provided by the role `acme-read-bucket-role`. This role
has the  policies `acme-read-bucket-policy` attached to it:

1. `acme-list-bucket-policy`
2. `acme-read-bucket-policy`
```json
         {
            "Sid": "WriteObjectsInBucket",
            "Effect": "Allow",
            "Action": ""s3:Get*"",
            "Resource": ["arn:aws:s3:::domino-acme-test-bucket/*"]
        }
```

Run the following code snippet:

In [ ]:
import boto3
import os
test_bucket='domino-acme-test-bucket'
starting_user = os.environ['DOMINO_STARTING_USERNAME']
read_bucket_profile_name='acme-read-bucket-role'
object_key = f'{starting_user}.txt'
session = boto3.session.Session(profile_name=read_bucket_profile_name)
s3_client = session.client('s3')
data = s3_client.get_object(Bucket=test_bucket, Key=object_key)
contents = data['Body'].read()
print(f'\n---Contents of the key {object_key}----\n')
print(contents.decode("utf-8"))

### Deleting a key from the bucket

Finally let us delete the newly created object

In [ ]:
import boto3
import os
test_bucket='domino-acme-test-bucket'
update_bucket_profile_name='acme-update-bucket-role'
starting_user = os.environ['DOMINO_STARTING_USERNAME']
object_key = f'{starting_user}.txt'
print(f'Deleting Key {object_key} from bucket {test_bucket}')

session = boto3.session.Session(profile_name=update_bucket_profile_name)
s3_client = session.client('s3')
s3_client.delete_object(Bucket=test_bucket, Key=object_key)
print('\nNow listing bucket:\n')
for key in s3_client.list_objects(Bucket=test_bucket)['Contents']:
    print(key)
